In [ ]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, GlobalAveragePooling2D, Dropout, Flatten
from tensorflow.keras.applications import VGG16

# SEED = 55

# np.random.seed = SEED
# tf.random.set_seed(SEED)


In [ ]:
train_dir = '/kaggle/input/100-bird-species/train'
val_dir = '/kaggle/input/100-bird-species/valid'
test_dir = '/kaggle/input/100-bird-species/test'

In [ ]:
BS = 64
img_size = 224

In [ ]:
train_datagen = ImageDataGenerator(rescale=1/255.,
                                   rotation_range=20,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   )

val_datagen = ImageDataGenerator(rescale=1/255.)

test_datagen = ImageDataGenerator(rescale=1/255.)

train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=(
                                                        img_size, img_size),
                                                    batch_size=BS,
                                                    shuffle=True,
                                                    class_mode='categorical')

validation_generator = val_datagen.flow_from_directory(val_dir,
                                                       target_size=(
                                                           img_size, img_size),
                                                       batch_size=BS,
                                                       shuffle=False,
                                                       class_mode='categorical')

test_generator = test_datagen.flow_from_directory(test_dir,
                                                  target_size=(
                                                      img_size, img_size),
                                                  batch_size=BS,
                                                  shuffle=False,
                                                  class_mode='categorical')

In [ ]:
labels = [k for k in train_generator.class_indices]
sample_generate = train_generator.__next__()

images = sample_generate[0]
titles = sample_generate[1]
plt.figure(figsize = (20 , 20))

for i in range(15):
    plt.subplot(5 , 5, i+1)
    plt.subplots_adjust(hspace = 0.3 , wspace = 0.3)
    plt.imshow(images[i])
    plt.title(f'Class: {labels[np.argmax(titles[i],axis=0)]}')
    plt.axis("off")

# Model without Trainable

In [ ]:
from tensorflow.keras.applications.resnet import ResNet101
base_model = ResNet101(include_top = False,
                       weights = 'imagenet',
                       input_shape = (img_size,img_size,3))

for layer in base_model.layers:
    layer.trainable = False

In [ ]:
for layer in base_model.layers:
    sp = " ------------------->" [len(layer.name)-9:]
    print(layer.name, sp, layer.trainable)

In [ ]:
inputs = tf.keras.Input(shape=(img_size,img_size, 3))
x = base_model(inputs, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(512, activation='relu')(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(256, activation='relu')(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(128, activation='relu')(x)
x = tf.keras.layers.Dropout(0.2)(x)
output = tf.keras.layers.Dense(515, activation='softmax')(x)

model = tf.keras.Model(inputs, output)
model.summary()

In [ ]:
def modelDetails(history, name):
    plt.figure(figsize=(10,4))
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title(name + ' Loss')
    plt.ylabel('loss')
    plt.xlabel('epochs')
    plt.legend(['Train','Val'], loc= 'upper left')
    plt.show()
    
    plt.figure(figsize=(10,3))
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title(name + ' Accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['Train', 'Val'], loc='upper left')
    plt.show()

In [ ]:
from tensorflow.keras.optimizers import Adam

model.compile(optimizer=Adam(learning_rate=0.0001),
             loss='categorical_crossentropy',
             metrics=['accuracy'])

history = model.fit(train_generator,
                    epochs=25,
                    verbose=1,
                    steps_per_epoch=len(train_generator),
                    validation_steps=len(validation_generator),
                    validation_data=validation_generator, 
                    use_multiprocessing=True,
                    workers=2,)

In [ ]:
results1 = model.evaluate(test_generator)

# Model with Tranable

In [ ]:
base_model2 = ResNet101(include_top = True,
                       weights = 'imagenet',
                       input_shape = (img_size,img_size,3))


In [ ]:
for layer in base_model2.layers[:-2]:
    layer.trainable = True


for layer in base_model2.layers:
    sp = " ------------------->" [len(layer.name)-9:]
    print(layer.name, sp, layer.trainable)

In [ ]:
# base_model2.summary()

In [ ]:
from tensorflow.keras.models import Model
newmodel = Model(base_model2.input, outputs= base_model2.layers[-2].output)

In [ ]:
newmodel.output.name

In [ ]:
inputs = tf.keras.Input(shape=(img_size,img_size, 3))
x = newmodel(inputs)
x = tf.keras.layers.Dense(512, activation='relu')(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(256, activation='relu')(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(128, activation='relu')(x)
x = tf.keras.layers.Dropout(0.2)(x)
output = tf.keras.layers.Dense(515, activation='softmax')(x)

model2 = tf.keras.Model(inputs, output)
model2.summary()

In [ ]:
model2.compile(optimizer=Adam(learning_rate=0.0001),
             loss='categorical_crossentropy',
             metrics=['accuracy'])

history2 = model2.fit(train_generator,
                      epochs=25,
                      verbose=1, 
                      steps_per_epoch=len(train_generator),
                      validation_steps=len(validation_generator),
                      validation_data=validation_generator, 
                      use_multiprocessing=True, 
                      workers=2,)

In [ ]:
results2 = model2.evaluate(test_generator)

In [ ]:
modelDetails(history, 'Model without trainable')

In [ ]:
modelDetails(history2, 'Model with trainable')